In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

from nbdt.model import SoftNBDT
from nbdt.models import wrn28_10_cifar10
from decompose import decompose

import numpy as np
import random
import matplotlib.pyplot as plt

from data_loader_cifar10 import get_train_valid_loader, get_test_loader
from pytorchtools import EarlyStopping, test_model, train_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

stty: 'standard input': Inappropriate ioctl for device


not enough values to unpack (expected 2, got 0)


# Load Data

In [2]:
batch_size = 128

trainloader, validloader = get_train_valid_loader("./data", batch_size, False, random_seed=0)
testloader = get_test_loader("./data", batch_size, random_seed=0)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Pretrained (WideResNet)

### Test Pretrained model

In [3]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
)

nParam = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parameters in model: {nParam}")

acc, carbontracker_full = test_model(model, testloader, resnet=True)
acc

Parameters in model: 36479194
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti. CPU with device(s) cpu:0.
CarbonTracker: INFO - Traceback (most recent call last):
  File "/home/borge/anaconda3/lib/python3.9/site-packages/carbontracker/emissions/intensity/intensity.py", line 62, in carbon_intensity
    carbon_intensity = fetcher.carbon_intensity(g_location, time_dur)
  File "/home/borge/anaconda3/lib/python3.9/site-packages/carbontracker/emissions/intensity/fetchers/energidataservice.py", line 19, in carbon_intensity
    ci = self._emission_current()
  File "/home/borge/anaconda3/lib/python3.9/site-packages/carbontracker/emissions/intensity/fetchers/energidataservice.py", line 42, in _emission_current
    raise exceptions.CarbonIntensityFetcherError(response.json())
carbontracker.exceptions.CarbonIntensityFetcherError: {'help': 'https://www.energidataservice.dk/guides/api-guides', 'success': False, 'error': {'info': {'orig': ['API has mov

0.9732

## Decomp pretrained model and test

In [6]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
).to(device)

factor = 4
model = decompose(model, factor, './Models/', include_params_and_factor=True)

10353190


In [4]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
).to(device)

factor = 4
model = decompose(model, factor, './Models/', include_params_and_factor=True)

n_epochs = 5
patience = 15
lr = 0.004
save_path = f'./Results/test_decomp_{factor=}.pt'

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

train_model(model, trainloader, validloader, optimizer, criterion, n_epochs, patience, save_path, resnet=True)
test_model(model, testloader, resnet=True)

/home/borge/anaconda3/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


10353190
[1/5] train_loss: 1.49055 valid_loss: 1.48147
Validation loss decreased (inf --> 1.481475).  Saving model ...
[2/5] train_loss: 1.47657 valid_loss: 1.47598
Validation loss decreased (1.481475 --> 1.475981).  Saving model ...
[3/5] train_loss: 1.47283 valid_loss: 1.47364
Validation loss decreased (1.475981 --> 1.473644).  Saving model ...
[4/5] train_loss: 1.47065 valid_loss: 1.47201
Validation loss decreased (1.473644 --> 1.472011).  Saving model ...
[5/5] train_loss: 1.46939 valid_loss: 1.47070
Validation loss decreased (1.472011 --> 1.470698).  Saving model ...
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti. CPU with device(s) cpu:0.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti. CPU with device(s) cpu:0.
CarbonTracker: INFO - Traceback (most recent call last):
  File "/home/borge/anaconda3/lib/python3.9/site-packages/carbontracker/emissions/intensity/intensity.py", line 62

(0.9672, <carbontracker.tracker.CarbonTracker at 0x7fb95c441bb0>)